# SnackTrack ML — Collaborative Filtering Analysis

This notebook explores user-recipe interaction data to understand the collaborative
filtering landscape for SnackTrack. We examine the interaction matrix structure,
user-user similarities, cold-start characteristics, and generate sample
recommendations to validate our approach before integrating into the production
hybrid recommender.

**Key questions:**
- How sparse is our user-recipe interaction matrix?
- Can we find meaningful user neighborhoods via cosine similarity?
- How many interactions does a user need before collaborative filtering becomes viable?
- What do sample recommendations look like?

In [ ]:
import sys
sys.path.insert(0, "..")

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics.pairwise import cosine_similarity
from scipy import sparse

from notebooks.utils.plot_helpers import setup_plot_style

setup_plot_style()

print("Imports ready.")

## 1. Load Interaction Data

We attempt to load user-recipe interactions from the SnackTrack database first.
If the database is unavailable or empty, we fall back to the Food.com dataset
which provides a rich set of user-recipe ratings for offline analysis.

In [ ]:
from notebooks.utils.data_loader import (
    load_interactions_from_db,
    load_kaggle_dataset,
)
from notebooks.utils.db_connect import get_connection

interactions_df = None
data_source = None

# ----- Attempt 1: Load from database -----
try:
    conn = get_connection()
    db_interactions = load_interactions_from_db(conn)
    conn.close()
    if not db_interactions.empty and len(db_interactions) >= 100:
        # Map interaction_value to a rating-like score
        interactions_df = db_interactions.rename(
            columns={"interaction_value": "rating"}
        )[["user_id", "recipe_id", "rating"]]
        data_source = "SnackTrack DB"
        print(f"Loaded {len(interactions_df)} interactions from the database.")
except Exception as e:
    print(f"Database unavailable: {e}")

# ----- Attempt 2: Fall back to Food.com dataset -----
if interactions_df is None:
    try:
        raw = load_kaggle_dataset("food_com_interactions")
        # Food.com columns: user_id, recipe_id, date, rating, review
        interactions_df = raw[["user_id", "recipe_id", "rating"]].copy()
        # Remove zero-ratings ("I haven't made this") to focus on actual feedback
        interactions_df = interactions_df[interactions_df["rating"] > 0]
        data_source = "Food.com (Kaggle)"
        print(f"Loaded {len(interactions_df)} interactions from Food.com dataset.")
    except FileNotFoundError:
        # ----- Attempt 3: Generate synthetic data for demonstration -----
        print("No dataset found. Generating synthetic interactions for demonstration.")
        rng = np.random.default_rng(42)
        n_users, n_recipes = 500, 2000
        n_interactions = 25000
        interactions_df = pd.DataFrame({
            "user_id": rng.integers(0, n_users, size=n_interactions),
            "recipe_id": rng.integers(0, n_recipes, size=n_interactions),
            "rating": rng.choice([1, 2, 3, 4, 5], size=n_interactions,
                                 p=[0.05, 0.10, 0.20, 0.35, 0.30]),
        })
        # Deduplicate (keep last rating per user-recipe pair)
        interactions_df = interactions_df.drop_duplicates(
            subset=["user_id", "recipe_id"], keep="last"
        ).reset_index(drop=True)
        data_source = "Synthetic"

print(f"\nData source: {data_source}")
print(f"Shape: {interactions_df.shape}")
print(f"Unique users:   {interactions_df['user_id'].nunique():,}")
print(f"Unique recipes: {interactions_df['recipe_id'].nunique():,}")
print(f"\nRating distribution:")
print(interactions_df["rating"].describe().round(2))
print(f"\nFirst 5 rows:")
interactions_df.head()

## 2. Build User-Recipe Interaction Matrix

We pivot the interaction data into a user-by-recipe matrix. For large datasets
(Food.com has ~200k users and ~200k recipes), we use a sparse matrix
representation to keep memory usage manageable. The sparsity percentage tells us
how much of the matrix is unfilled -- typically >99% for recommendation datasets.

In [ ]:
# Build ID-to-index mappings
user_ids = sorted(interactions_df["user_id"].unique())
recipe_ids = sorted(interactions_df["recipe_id"].unique())

user_to_idx = {uid: i for i, uid in enumerate(user_ids)}
recipe_to_idx = {rid: i for i, rid in enumerate(recipe_ids)}
idx_to_user = {i: uid for uid, i in user_to_idx.items()}
idx_to_recipe = {i: rid for rid, i in recipe_to_idx.items()}

n_users = len(user_ids)
n_recipes = len(recipe_ids)

# Build sparse matrix (COO format, then convert to CSR for fast row slicing)
row_indices = interactions_df["user_id"].map(user_to_idx).values
col_indices = interactions_df["recipe_id"].map(recipe_to_idx).values
values = interactions_df["rating"].values.astype(np.float32)

interaction_matrix = sparse.csr_matrix(
    (values, (row_indices, col_indices)),
    shape=(n_users, n_recipes),
)

total_cells = n_users * n_recipes
filled_cells = interaction_matrix.nnz
sparsity = 1.0 - (filled_cells / total_cells)

print(f"Interaction matrix shape: {interaction_matrix.shape}")
print(f"Non-zero entries:         {filled_cells:,}")
print(f"Total cells:              {total_cells:,}")
print(f"Sparsity:                 {sparsity:.4%}")
print(f"Memory (sparse CSR):      {interaction_matrix.data.nbytes / 1024:.1f} KB")
print(f"Memory (dense equiv):     {total_cells * 4 / (1024**2):.1f} MB")

## 3. Sparsity Visualization

Visualizing the matrix structure helps us understand how interactions are
distributed. We examine:
1. A spy plot showing the non-zero pattern of a subset of the matrix
2. Histograms of interaction counts per user and per recipe (on log scale)

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# --- Spy plot of a subset (top 100 users x first 200 recipes) ---
subset_rows = min(100, n_users)
subset_cols = min(200, n_recipes)
subset = interaction_matrix[:subset_rows, :subset_cols].toarray()

axes[0].spy(subset, markersize=1, aspect="auto", color="#4CAF50")
axes[0].set_title(f"Interaction Pattern\n(top {subset_rows} users x {subset_cols} recipes)")
axes[0].set_xlabel("Recipe Index")
axes[0].set_ylabel("User Index")

# --- Interactions per user (log scale) ---
interactions_per_user = np.array(interaction_matrix.getnnz(axis=1)).flatten()
axes[1].hist(interactions_per_user, bins=50, color="#2196F3", alpha=0.7, edgecolor="white")
axes[1].set_yscale("log")
axes[1].set_title("Interactions per User")
axes[1].set_xlabel("Number of Interactions")
axes[1].set_ylabel("Count (log scale)")
axes[1].axvline(np.median(interactions_per_user), color="#F44336", linestyle="--",
                label=f"Median: {np.median(interactions_per_user):.0f}")
axes[1].legend()

# --- Interactions per recipe (log scale) ---
interactions_per_recipe = np.array(interaction_matrix.getnnz(axis=0)).flatten()
axes[2].hist(interactions_per_recipe, bins=50, color="#FF9800", alpha=0.7, edgecolor="white")
axes[2].set_yscale("log")
axes[2].set_title("Interactions per Recipe")
axes[2].set_xlabel("Number of Interactions")
axes[2].set_ylabel("Count (log scale)")
axes[2].axvline(np.median(interactions_per_recipe), color="#F44336", linestyle="--",
                label=f"Median: {np.median(interactions_per_recipe):.0f}")
axes[2].legend()

plt.tight_layout()
plt.show()

print(f"Interactions per user  — mean: {interactions_per_user.mean():.1f}, "
      f"median: {np.median(interactions_per_user):.0f}, "
      f"max: {interactions_per_user.max()}")
print(f"Interactions per recipe — mean: {interactions_per_recipe.mean():.1f}, "
      f"median: {np.median(interactions_per_recipe):.0f}, "
      f"max: {interactions_per_recipe.max()}")

## 4. User-User Similarity

Collaborative filtering relies on finding users with similar taste profiles.
We compute cosine similarity between the top-50 most active users to examine
whether meaningful clusters emerge. More active users typically produce more
reliable similarity scores.

In [ ]:
# Select top-50 most active users
top_k = 50
most_active_idx = np.argsort(interactions_per_user)[::-1][:top_k]
active_matrix = interaction_matrix[most_active_idx].toarray()

# Compute cosine similarity
user_sim = cosine_similarity(active_matrix)

fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# --- Heatmap ---
sns.heatmap(
    user_sim,
    ax=axes[0],
    cmap="YlOrRd",
    xticklabels=False,
    yticklabels=False,
    vmin=0,
    vmax=1,
    cbar_kws={"label": "Cosine Similarity"},
)
axes[0].set_title(f"User-User Cosine Similarity\n(top {top_k} most active users)")
axes[0].set_xlabel("User")
axes[0].set_ylabel("User")

# --- Distribution of off-diagonal similarity scores ---
mask = np.triu(np.ones_like(user_sim, dtype=bool), k=1)
off_diag = user_sim[mask]

axes[1].hist(off_diag, bins=50, color="#9C27B0", alpha=0.7, edgecolor="white")
axes[1].set_title("Distribution of Pairwise Similarities")
axes[1].set_xlabel("Cosine Similarity")
axes[1].set_ylabel("Count")
axes[1].axvline(off_diag.mean(), color="#F44336", linestyle="--",
                label=f"Mean: {off_diag.mean():.3f}")
axes[1].axvline(np.median(off_diag), color="#2196F3", linestyle="--",
                label=f"Median: {np.median(off_diag):.3f}")
axes[1].legend()

plt.tight_layout()
plt.show()

print(f"Similarity stats (off-diagonal, top-{top_k} users):")
print(f"  Mean:   {off_diag.mean():.4f}")
print(f"  Median: {np.median(off_diag):.4f}")
print(f"  Std:    {off_diag.std():.4f}")
print(f"  Min:    {off_diag.min():.4f}")
print(f"  Max:    {off_diag.max():.4f}")
print(f"  Pairs with sim > 0.1: {(off_diag > 0.1).sum()} / {len(off_diag)}")

## 5. Cold-Start Analysis

A critical challenge in collaborative filtering is the **cold-start problem**:
new users with few interactions cannot be reliably matched to similar users.

Here we simulate cold-start conditions by selecting users with varying numbers
of interactions and measuring how many "effective neighbors" they have
(i.e., users with cosine similarity > 0.1). This informs our production
`COLD_START_THRESHOLD` setting.

In [ ]:
# Build a reference set from the top-200 most active users
ref_k = min(200, n_users)
ref_idx = np.argsort(interactions_per_user)[::-1][:ref_k]
ref_matrix = interaction_matrix[ref_idx].toarray()

# Interaction count thresholds to test
thresholds = [1, 3, 5, 10, 20, 50]
similarity_cutoff = 0.1

results = []
n_trials = 20  # number of users to sample per threshold

rng = np.random.default_rng(42)

for threshold in thresholds:
    # Find users with approximately this many interactions
    eligible = np.where(
        (interactions_per_user >= threshold) &
        (interactions_per_user < threshold * 3)
    )[0]

    if len(eligible) == 0:
        # If no users match the range, take users closest to the threshold
        eligible = np.argsort(np.abs(interactions_per_user - threshold))[:n_trials]

    sample_idx = rng.choice(eligible, size=min(n_trials, len(eligible)), replace=False)
    
    neighbor_counts = []
    for idx in sample_idx:
        user_vec = interaction_matrix[idx].toarray()
        sims = cosine_similarity(user_vec, ref_matrix)[0]
        # Exclude self if in reference set
        effective_neighbors = (sims > similarity_cutoff).sum()
        neighbor_counts.append(effective_neighbors)

    results.append({
        "interaction_count": threshold,
        "mean_neighbors": np.mean(neighbor_counts),
        "std_neighbors": np.std(neighbor_counts),
        "median_neighbors": np.median(neighbor_counts),
        "min_neighbors": np.min(neighbor_counts),
        "max_neighbors": np.max(neighbor_counts),
    })

results_df = pd.DataFrame(results)
print(results_df.to_string(index=False))

# --- Plot ---
fig, ax = plt.subplots(figsize=(10, 6))

ax.errorbar(
    results_df["interaction_count"],
    results_df["mean_neighbors"],
    yerr=results_df["std_neighbors"],
    marker="o",
    color="#4CAF50",
    linewidth=2,
    markersize=8,
    capsize=5,
    label="Mean +/- Std",
)
ax.plot(
    results_df["interaction_count"],
    results_df["median_neighbors"],
    marker="s",
    color="#FF9800",
    linewidth=2,
    markersize=6,
    linestyle="--",
    label="Median",
)

ax.set_xlabel("User Interaction Count")
ax.set_ylabel(f"Effective Neighbors (sim > {similarity_cutoff})")
ax.set_title("Cold-Start Analysis: Effective Neighbors vs. Interaction Count")
ax.legend()
ax.grid(True, alpha=0.3)

# Mark the production cold-start threshold (from SnackTrack config)
ax.axvline(5, color="#F44336", linestyle=":", alpha=0.7,
           label="Production threshold (5)")
ax.legend()

plt.tight_layout()
plt.show()

## 6. Recommendation Generation

We demonstrate the full collaborative filtering pipeline for a sample user:
1. Find the K most similar users (by cosine similarity)
2. Aggregate their ratings weighted by similarity
3. Exclude recipes the target user has already rated
4. Return the top-10 recommended recipes

This mirrors the logic in `app/recommender/collaborative.py`.

In [ ]:
def collaborative_recommend(user_idx, interaction_mat, k_neighbors=20, top_n=10):
    """Generate collaborative filtering recommendations for a single user.
    
    Parameters
    ----------
    user_idx : int
        Index of the target user in the interaction matrix.
    interaction_mat : sparse matrix
        User-recipe interaction matrix (CSR format).
    k_neighbors : int
        Number of similar users to consider.
    top_n : int
        Number of recommendations to return.
    
    Returns
    -------
    recommendations : list of (recipe_idx, predicted_score)
    neighbor_sims : array of similarity scores for the chosen neighbors
    """
    user_vec = interaction_mat[user_idx].toarray().flatten()
    
    # Compute similarity to all other users
    all_sims = cosine_similarity(
        user_vec.reshape(1, -1),
        interaction_mat.toarray()
    )[0]
    
    # Zero out self-similarity
    all_sims[user_idx] = 0
    
    # Select top-K neighbors with positive similarity
    neighbor_idx = np.argsort(all_sims)[::-1][:k_neighbors]
    neighbor_sims = all_sims[neighbor_idx]
    
    # Keep only neighbors with positive similarity
    positive_mask = neighbor_sims > 0
    neighbor_idx = neighbor_idx[positive_mask]
    neighbor_sims = neighbor_sims[positive_mask]
    
    if len(neighbor_idx) == 0:
        return [], np.array([])
    
    # Weighted aggregation of neighbor ratings
    neighbor_matrix = interaction_mat[neighbor_idx].toarray()
    weighted_scores = neighbor_sims @ neighbor_matrix  # (k,) @ (k, n_recipes) -> (n_recipes,)
    sim_sum = neighbor_sims.sum()
    if sim_sum > 0:
        weighted_scores /= sim_sum  # normalize by total similarity
    
    # Exclude recipes the user has already rated
    already_rated = np.where(user_vec > 0)[0]
    weighted_scores[already_rated] = 0
    
    # Get top-N
    top_recipe_idx = np.argsort(weighted_scores)[::-1][:top_n]
    recommendations = [
        (int(ridx), float(weighted_scores[ridx]))
        for ridx in top_recipe_idx
        if weighted_scores[ridx] > 0
    ]
    
    return recommendations, neighbor_sims


# Pick a sample user who has a reasonable number of interactions
target_interaction_count = 15
candidate_users = np.where(
    (interactions_per_user >= target_interaction_count) &
    (interactions_per_user <= target_interaction_count * 5)
)[0]

if len(candidate_users) == 0:
    # Fallback: pick the user closest to the target count
    sample_user_idx = np.argmin(np.abs(interactions_per_user - target_interaction_count))
else:
    sample_user_idx = candidate_users[0]

sample_user_id = idx_to_user[sample_user_idx]
sample_user_ratings = interactions_df[interactions_df["user_id"] == sample_user_id]

print(f"Sample user: {sample_user_id}")
print(f"Number of interactions: {interactions_per_user[sample_user_idx]}")
print(f"\nUser's existing ratings:")
print(sample_user_ratings.head(10).to_string(index=False))

# Generate recommendations
recs, neighbor_sims = collaborative_recommend(
    sample_user_idx, interaction_matrix, k_neighbors=20, top_n=10
)

print(f"\n{'='*60}")
print(f"Top-10 Collaborative Filtering Recommendations")
print(f"{'='*60}")
print(f"{'Rank':<6} {'Recipe ID':<15} {'Predicted Score':<15}")
print(f"{'-'*6} {'-'*15} {'-'*15}")

for rank, (ridx, score) in enumerate(recs, 1):
    recipe_id = idx_to_recipe[ridx]
    print(f"{rank:<6} {str(recipe_id):<15} {score:<15.4f}")

print(f"\nNeighbors used: {len(neighbor_sims)}")
if len(neighbor_sims) > 0:
    print(f"Neighbor similarity range: [{neighbor_sims.min():.4f}, {neighbor_sims.max():.4f}]")

## 7. Key Findings

Summary statistics and observations from the collaborative filtering analysis.

In [ ]:
print("=" * 65)
print("  COLLABORATIVE FILTERING ANALYSIS — KEY FINDINGS")
print("=" * 65)
print()
print(f"Data source:              {data_source}")
print(f"Total interactions:       {len(interactions_df):,}")
print(f"Unique users:             {n_users:,}")
print(f"Unique recipes:           {n_recipes:,}")
print(f"Matrix sparsity:          {sparsity:.4%}")
print(f"Mean interactions/user:   {interactions_per_user.mean():.1f}")
print(f"Median interactions/user: {np.median(interactions_per_user):.0f}")
print(f"Mean interactions/recipe: {interactions_per_recipe.mean():.1f}")
print()
print("--- User Similarity (top-50 active users) ---")
print(f"Mean cosine similarity:   {off_diag.mean():.4f}")
print(f"Pairs with sim > 0.1:     {(off_diag > 0.1).sum()} / {len(off_diag)}")
print()
print("--- Cold-Start Insights ---")
for _, row in results_df.iterrows():
    print(f"  {int(row['interaction_count']):3d} interactions -> "
          f"{row['mean_neighbors']:.1f} effective neighbors (avg)")
print()
print("--- Observations ---")
print("1. The interaction matrix is extremely sparse, which is typical")
print("   for recommendation systems. Sparse matrices are essential.")
print("2. User-user similarity increases with interaction count,")
print("   confirming collaborative filtering improves with more data.")
print("3. Cold-start users (<5 interactions) have very few effective")
print("   neighbors, justifying the production COLD_START_THRESHOLD.")
print("4. The hybrid recommender should weight content-based and")
print("   knowledge-based signals more heavily for cold-start users,")
print("   transitioning to collaborative filtering as data accumulates.")
print()
print("These findings directly inform the weight blending in")
print("app/recommender/hybrid.py.")